# Install Dependencies

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

# Import Libraries

In [ ]:
import os
import sqlite3
import json
import google.generativeai as genai
from google.colab import drive
import gradio as gr
from typing import List, Dict, Any
import traceback

# Acces and Connect to the Database

In [ ]:
# Mount Google Drive to access the database
drive.mount('/content/drive')

# Database connection
DB_PATH = "/content/drive/My Drive/work@home4/Northwind.db"

def get_db_connection():
    """Create and return a connection to the SQLite database."""
    try:
        conn = sqlite3.connect(DB_PATH, timeout=10)
        conn.row_factory = sqlite3.Row  # This enables column access by name
        return conn
    except sqlite3.Error as e:
        print(f"Database connection error: {e}")
        return None

Mounted at /content/drive


# Set up and Configure Gemini Api

In [ ]:
# Set up the Google API key
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure the Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

# Schemas

In [ ]:
def get_table_schema():
    """Extract the schema from the database to inform the model."""
    try:
        conn = get_db_connection()
        if not conn:
            return "Could not connect to database to extract schema."

        cursor = conn.cursor()

        # Get a list of all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        schema = {}
        for table in tables:
            table_name = table[0]
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()

            schema[table_name] = {
                "columns": [{"name": col[1], "type": col[2]} for col in columns],
                "primary_key": next((col[1] for col in columns if col[5] == 1), None)
            }

        conn.close()
        return json.dumps(schema, indent=2)
    except Exception as e:
        print(f"Error getting schema: {e}")
        traceback.print_exc()
        return json.dumps({"error": str(e)})

# Get the schema once at the beginning
print("Fetching database schema...")
DB_SCHEMA = get_table_schema()
print("Schema fetched successfully!")


# Define the JSON schema for structured output
json_schema = {
    "type": "object",
    "properties": {
        "sql_query": {
            "type": "string",
            "description": "The SQL query to execute on the database"
        },
        "explanation": {
            "type": "string",
            "description": "A brief explanation of what the SQL query does"
        }
    },
    "required": ["sql_query", "explanation"]
}

Fetching database schema...
Schema fetched successfully!


# Execute Sql Query Method

In [ ]:
def execute_sql_query(sql_query: str) -> List[Dict[str, Any]]:
    """Execute the given SQL query and return the results as a list of dictionaries."""
    conn = get_db_connection()
    if not conn:
        return [{"error": "Could not connect to database"}]

    try:
        cursor = conn.cursor()
        cursor.execute(sql_query)

        # Convert the results to a list of dictionaries
        columns = [col[0] for col in cursor.description]
        results = [dict(zip(columns, row)) for row in cursor.fetchall()]

        conn.close()
        return results
    except sqlite3.Error as e:
        conn.close()
        return [{"error": f"SQL error: {e}", "query": sql_query}]

# Generate Sql Method

In [ ]:
def generate_sql_from_nl(user_query: str) -> Dict[str, Any]:
    """Generate SQL from natural language using Gemini API with structured output."""
    try:
        # Simplified schema for the prompt to reduce token usage
        simplified_schema = json.loads(DB_SCHEMA)
        schema_prompt = "Tables and their columns:\n"
        for table, details in simplified_schema.items():
            columns = [col['name'] for col in details['columns']]
            schema_prompt += f"- {table}: {', '.join(columns)}\n"

        system_prompt =f"""You are an expert SQL assistant that helps generate SQL queries for a Northwind company database. Your sole goal is to understand the user's request and generate a valid SQL query for the Northwind database.
        The database contains the following tables and columns:
        {schema_prompt}

        **Do:**

        * Generate valid SQLite SQL queries based on user requests
        * Use only tables and columns that exist in the schema
        * Handle requests in both English and Turkish
        * Ensure proper JOIN syntax when combining tables
        * Keep SQL queries simple and efficient
        * Always return responses in the required JSON format

        **Do not:**

        * Use tables or columns that don't exist in the schema
        * Provide alternative solutions unless the user specifically asks
        * Include unnecessary complexity in queries
        * Respond with anything other than the required JSON format

        **Example 1:**
        **You:**  "Is there anything else I can help you with today?"
        **Customer:** "Show me all customers from Germany"
        **You:** "
  "sql_query": "SELECT * FROM Customers WHERE Country = 'Germany';",
  "explanation": "This query selects all customer records where the Country field equals 'Germany'."
"
        **Example 2:**
        **You:**  "Is there anything else I can help you with today?"
        **Customer:** "En pahalı 5 ürünü listele"
        **You:** "
  "sql_query": "SELECT ProductName, Price FROM Products ORDER BY Price DESC LIMIT 5;",
  "explanation": "Bu sorgu, ürünleri fiyatlarına göre azalan sırada sıralar ve en pahalı 5 ürünün adını ve fiyatını getirir."
"

        **Example 3:**
        **You:**  "Is there anything else I can help you with today?"
        **Customer:** "Which employee handled the most orders?"
        **You:** "
  "sql_query": "SELECT e.EmployeeID, e.FirstName, e.LastName, COUNT(o.OrderID) as OrderCount FROM Employees e JOIN Orders o ON e.EmployeeID = o.EmployeeID GROUP BY e.EmployeeID ORDER BY OrderCount DESC LIMIT 1;",
  "explanation": "This query counts the orders handled by each employee and returns the employee who handled the most orders."
"

        **Example 4:**
        **You:**  "Is there anything else I can help you with today?"
        **Customer:** "Hangi tedarikçi en çok ürün sağlıyor?"
        **You:** "
  "sql_query": "SELECT s.SupplierID, s.SupplierName, COUNT(p.ProductID) as ProductCount FROM Suppliers s JOIN Products p ON s.SupplierID = p.SupplierID GROUP BY s.SupplierID ORDER BY ProductCount DESC LIMIT 1;",
  "explanation": "Bu sorgu, her tedarikçinin sağladığı ürün sayısını hesaplar ve en çok ürün sağlayan tedarikçiyi döndürür."
"

        **Note**
        Continue the interaction according to the language used by the user.
        *If the user writes in Turkish, respond with a Turkish explanation.
        *If the user writes in English, respond with an English explanation.

        """

        generation_config = {
          "temperature": 0.1,
           "top_p": 0.95,
            "top_k": 64,
            "max_output_tokens": 8192,
            "response_mime_type": "application/json",
             }

        model = genai.GenerativeModel(
            model_name="gemini-1.5-flash-latest",
            generation_config=generation_config,
            system_instruction=system_prompt
        )

        print("Sending request to Gemini API...")
        response = model.generate_content(
            user_query,
            generation_config={"response_schema": json_schema}
        )
        print("Received response from Gemini API!")

        try:
            result = json.loads(response.text)
            return result
        except json.JSONDecodeError:
            print(f"Failed to parse JSON. Raw response: {response.text}")
            return {
                "sql_query": "",
                "explanation": "Failed to parse response from AI model. Please try again."
            }
    except Exception as e:
        print(f"Error in generate_sql_from_nl: {e}")
        traceback.print_exc()
        return {
            "sql_query": "",
            "explanation": f"Error communicating with AI model: {str(e)}. Please try again."
        }

# Chat Interface Method

In [ ]:
def create_chat_interface():
    """Create and launch the Gradio chat interface."""
    with gr.Blocks(css=".chatbot-container { max-width: 500px; margin: auto; } footer { visibility: hidden; }") as demo:
        gr.Markdown("# Northwind DB Chatbot")
        gr.Markdown("İngilizce veya Türkçe olarak şirket veritabanı hakkında sorular sorun!")

        with gr.Row():
            with gr.Column(scale=3):
                # Initialize chatbot with a welcome message
                chatbot = gr.Chatbot(value=[
                    (None, "Merhaba! Northwind veritabanı hakkında sorularınızı yanıtlamak için buradayım. Size nasıl yardımcı olabilirim?")
                ], height=500)

                with gr.Row():
                    msg = gr.Textbox(placeholder="Sorunuzu buraya yazın... (örn: 'En pahalı 5 ürün nedir?')", label="Sorunuz")
                    send_btn = gr.Button("Gönder")

                clear = gr.Button("Temizle")

            with gr.Column(scale=1):
                gr.Markdown("### Örnek Sorular")
                example_btn1 = gr.HTML("Almanya'daki müşteriler kimler?")
                example_btn2 = gr.HTML("En pahalı 5 ürünü listele")
                example_btn3 = gr.HTML("Hangi tedarikçi en çok ürün sağlıyor?")
                example_btn4 = gr.HTML("Alice Mutton ürününün fiyatı nedir?")



        def user(user_message, history):
            """Handle user input and update the chat history."""
            return "", history + [(user_message, None)]

        def bot(history):
            """Process user query and generate response."""
            user_message = history[-1][0]
            history = history[:-1]

            try:
                # Generate SQL from natural language
                response = generate_sql_from_nl(user_message)

                sql_query = response.get("sql_query", "")
                explanation = response.get("explanation", "")

                if not sql_query:
                    history.append((user_message, "Sorgumuz için geçerli bir SQL oluşturamadım. Lütfen Northwind DB ile ilgili sorular sorun?"))
                    return history

                # Execute the SQL query
                query_results = execute_sql_query(sql_query)

                if len(query_results) == 1 and "error" in query_results[0]:
                    error_message = query_results[0]["error"]
                    result = f"Sorgu çalıştırılırken hata: {error_message}\n\nDenenen SQL: {sql_query}"
                else:
                    # Format the results
                    formatted_results = json.dumps(query_results, indent=2, ensure_ascii=False)

                    # Create a full response with explanation, SQL, and results
                    result = f"""**Anladığım:**
                    {explanation}
                    **SQL Sorgusu:**
                    ```sql
                    {sql_query}
                    ```
                    **Sonuçlar:**
                    ```json
                    {formatted_results if query_results else "Sonuç bulunamadı."}
                    ```
                    {len(query_results)} kayıt bulundu."""

                history.append((user_message, result))
            except Exception as e:
                history.append((user_message, f"Beklenmedik bir hata oluştu: {str(e)}"))
                traceback.print_exc()

            return history

        def clear_chat():
            """Clear the chat history."""
            return []

        # Connect the interface components
        send_btn.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
            bot, chatbot, chatbot
        )

        msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
            bot, chatbot, chatbot
        )

        clear.click(clear_chat, None, chatbot)

    return demo

# Create and launch the chat interface
print("Arayüz başlatılıyor...")
chat_interface = create_chat_interface()
chat_interface.launch(share=True, debug=True)

Arayüz başlatılıyor...


<ipython-input-9-0cb446bfb812>:10: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value=[


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d0a296f8b1adc1b804.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Sending request to Gemini API...
Received response from Gemini API!
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f1bb95ac51d1dc0cda.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://d0a296f8b1adc1b804.gradio.live
